In [1]:
# %load Main.py
import numpy as np
import tensorflow as tf
import PreProcess as PrePro
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date

from keras.optimizers import Adam, SGD,RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

import sklearn
import math
import unet_model
import Data_Load
import unet_info
import common
import pandas as pd
import logging
import argparse
import time
import segnet
import maskrcnn
import os
import openpyxl

Using TensorFlow backend.


In [ ]:
#T sne
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from keras.utils import np_utils

In [ ]:
def plot_TSNE(features,label,z,data_type,path):
    pca = PCA(n_components=2)# 总的类别
    nsamples, nx, ny, nz= features.shape 
    x = features.reshape([nsamples,nx*ny*nz])
    pca_result = pca.fit_transform(x)
    print('Variance PCA: {}'.format(np.sum(pca.explained_variance_ratio_)))

    #Run T-SNE on the PCA features.
    tsne = TSNE(n_components=2, verbose = 1)
    tsne_results = tsne.fit_transform(pca_result[:50])
    #-------------------------------可视化--------------------------------
    y_test_cat = np_utils.to_categorical(label[:50], num_classes = 2)# 总的类别
    color_map = np.argmax(y_test_cat, axis=1)
    plt.figure(figsize=(10,10))
    for cl in range(2):# 总的类别
        indices = np.where(color_map==cl)
        indices = indices[0]
        plt.scatter(tsne_results[indices,0], tsne_results[indices, 1], label=cl)
    plt.legend()
    plt.title(str(z)+'_'+data_type+'_T-sne')
    plt.show()
    plt.savefig(path+"/"+str(z)+'_'+data_type+'_T-sne.png')

In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
unet_info.begin()

GPU setting completed.
Tensorflow version: 1.10.0
Keras version 2.2.0
Dim ordering: tf


In [3]:
subseq =4096
#dbName  = "SepsisDataset"
dbName  = "SepsisDataset_random"
print("Load data")
X_train, X_test, Y_train, Y_test, N_FEATURES, classes = Data_Load.load_data(dbName,subseq)
#get dense prediction results with overlap(transformed win data label's ground truth)
# y_test_resh = y_test.reshape(y_test.shape[0], y_test.shape[2], -1)
# y_test_resh_argmax = np.argmax(y_test_resh, axis=2)
# labels_test_unary = y_test_resh_argmax.reshape(y_test_resh_argmax.size)
# file_labels_test_unary = 'labels_gd_'+args.dataset+'_'+str(subseq)+'0317.npy'
# np.save(file_labels_test_unary,labels_test_unary)
print("Finish Load data")

Load data
radom
(154800, 1, 4096, 1)
(50400, 1, 4096, 1)
SepsisA2B2_NorAllday
Finish Load data


In [4]:
## 10-flod cross validation###
epochs = 45 #50
batch_size = 64 #64 #128
optim_type = 'adam'
learning_rate = 0.02 # 0.05 # 0.01 #0.02
sum_time = 0
net = 'unet'
block = '5'

In [ ]:
nowtime = time.strftime("%m%d_%H%M",time.localtime())
s_path = 'Result/'+str(subseq)
print('Patient subsep Floder created') if os.path.isdir(s_path) else os.mkdir(s_path)
rf_path = s_path + '/'+nowtime + '_'+ str(subseq)
print('Patient Floder created') if os.path.isdir(rf_path) else os.mkdir(rf_path)

In [5]:
rf_path = 'Result/4096/0406_1603_4096'
nowtime = '0406_1603'
valcvscores =[[0.0162192333235769 , 0.993811761],[0.0242311698568682,0.993313653740156],
              [0.0216753581516536,0.993578849251572], [0.0610394489308891,0.980411279263114],
              [0.019811101,0.990566599],[0.019553627,0.993653495]]
testcvscores =[[2.3045928918747673 , 0.6679142737010169 ],[2.847108883479285,0.6585575648716517],
              [2.192989231215583,0.6919554695250496], [1.8850630738621665,0.6757591998387897],
              [2.696897084977891,0.6662840053013392],[2.61109272321065,0.6860669671921503]]
#valcvscores  = []
#testcvscores = []
print(valcvscores)
print(testcvscores)

[[0.0162192333235769, 0.993811761], [0.0242311698568682, 0.993313653740156], [0.0216753581516536, 0.993578849251572], [0.0610394489308891, 0.980411279263114], [0.019811101, 0.990566599], [0.019553627, 0.993653495]]
[[2.3045928918747673, 0.6679142737010169], [2.847108883479285, 0.6585575648716517], [2.192989231215583, 0.6919554695250496], [1.8850630738621665, 0.6757591998387897], [2.696897084977891, 0.6662840053013392], [2.61109272321065, 0.6860669671921503]]


In [ ]:
ksize = 10
pice = int(X_train.shape[0]/ ksize)
print('k-fold Floder created') if os.path.isdir(rf_path +'/k_flod') else os.mkdir(rf_path+'/k_flod')
for k in range(6,ksize):
    fp = open(rf_path+'/Result_'+nowtime+'.txt','a')
    trainX = X_train[np.r_[0:k*pice,(k+1)*pice:X_train.shape[0]]]
    trainY = Y_train[np.r_[0:k*pice,(k+1)*pice:X_train.shape[0]]]
    valX = X_train[np.r_[k*pice:(k+1)*pice]]
    valY = Y_train[np.r_[k*pice:(k+1)*pice]]
    print("\n",file=fp)
    print(str(k)+" Running...")
    print("\n",file=fp)
    print(str(k)+" Running...",file=fp)
    # create model
    Kmodel = unet_model.ZF_UNET_224(subseq=subseq, filters=32, INPUT_CHANNELS=N_FEATURES, OUTPUT_MASK_CHANNELS=classes)
    # Compile model
    optim = Adam(lr=learning_rate)
    Kmodel.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    
    # checkpoint
    lr_reducer = ReduceLROnPlateau(monitor='val_loss',factor=np.sqrt(0.1),cooldown=0,patience=10, min_lr=1e-12)
    filepath=rf_path+'/k_flod/'+str(k)+'_'+nowtime+'_'+"weights.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss',mode='min',verbose=1, save_best_only=True)   
    early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, mode='auto',baseline=None)
    # save best weight model
    callbacks = [lr_reducer, early_stopper, checkpoint]
    
    # Fit the model
    khistor =Kmodel.fit(trainX, trainY,validation_data=(valX, valY),batch_size=batch_size, epochs=epochs, callbacks=callbacks)
    '''
    #-------------------------------获取模型最后一层的数据--------------------------------
    #-------------------------------與PCA,tSNE降维分析-------------------------------------
    trucated_model=unet_model.create_truncate_model(Kmodel,subseq=subseq,filters=32,INPUT_CHANNELS=N_FEATURES,OUTPUT_MASK_CHANNELS=classes)
    val_hidden_features = trucated_model.predict(valX)
    plot_TSNE(val_hidden_features,valY,k,'validation_set',rf_path)
    del val_hidden_features
    
    plot_TSNE(test_hidden_features,Y_test,k,'test_set',rf_path)
    test_hidden_features = trucated_model.predict(X_test)
    del test_hidden_features
    '''
    # evaluate the model
    valKscores = Kmodel.evaluate(valX, valY, verbose=0)
    print("%s: %.2f%%" % (Kmodel.metrics_names[1], valKscores[1]*100), file=fp)
    print("%s: %.2f%%" % (Kmodel.metrics_names[0], valKscores[0]), file=fp)
    valcvscores.append(valKscores)

   
    testKscores = Kmodel.evaluate(X_test, Y_test, batch_size=batch_size)
    print("test lost:" + str(testKscores[0]) + "   test accuracy:" + str(testKscores[1]) + '\n',file=fp)
    testcvscores.append(testKscores)
    
    # flod save
    file_model = rf_path+'/k_flod/'+str(k)+'_Model_'+nowtime + '_'+ str(subseq) +'_' + str(batch_size) + '_' + optim_type + '.h5py'
    Kmodel.save(file_model)
    file_weight = rf_path+'/k_flod/'+str(k)+'_Weight_'+nowtime + '_'+ str(subseq) +'_' + str(batch_size) + '_' + optim_type + '.h5py'
    Kmodel.save_weights(file_weight)
    from openpyxl import load_workbook
    # convert the history.history dict to a pandas DataFrame:     
    khist_df = pd.DataFrame(khistor.history) 
    # save to csv: 
    if os.path.isfile(rf_path+'/'+nowtime+'_history.xlsx'):
        book = load_workbook(rf_path+'/'+nowtime+'_history.xlsx')     
        writer = pd.ExcelWriter(rf_path+'/'+nowtime+'_history.xlsx', engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    else:
        writer = pd.ExcelWriter(rf_path+'/'+nowtime+'_history.xlsx')
        
    khist_df.to_excel(writer,str(k)+'_flod')
    writer.save()
    fp.close()
    
    del trainX,trainY, valX, valY, Kmodel,khist_df

k-fold Floder created
6 Running...
Train on 139320 samples, validate on 15480 samples
Epoch 1/45
139320/139320 [==============================] - 655s 5ms/step - loss: 0.2867 - acc: 0.8618 - val_loss: 0.1604 - val_acc: 0.9431

Epoch 00001: val_loss improved from inf to 0.16038, saving model to Result/4096/0406_1603_4096/k_flod/6_0406_1603_weights.best.hdf5
Epoch 2/45
139320/139320 [==============================] - 625s 4ms/step - loss: 0.0782 - acc: 0.9687 - val_loss: 0.1157 - val_acc: 0.9648

Epoch 00002: val_loss improved from 0.16038 to 0.11566, saving model to Result/4096/0406_1603_4096/k_flod/6_0406_1603_weights.best.hdf5
Epoch 3/45
139320/139320 [==============================] - 624s 4ms/step - loss: 0.0542 - acc: 0.9780 - val_loss: 0.0809 - val_acc: 0.9665

Epoch 00003: val_loss improved from 0.11566 to 0.08095, saving model to Result/4096/0406_1603_4096/k_flod/6_0406_1603_weights.best.hdf5
Epoch 4/45
139320/139320 [==============================] - 623s 4ms/step - loss: 0.044

139320/139320 [==============================] - 618s 4ms/step - loss: 0.0104 - acc: 0.9959 - val_loss: 0.0297 - val_acc: 0.9914

Epoch 00038: val_loss did not improve from 0.01686
Epoch 39/45
139320/139320 [==============================] - 617s 4ms/step - loss: 0.0111 - acc: 0.9956 - val_loss: 0.0189 - val_acc: 0.9941

Epoch 00039: val_loss did not improve from 0.01686
Epoch 40/45
139320/139320 [==============================] - 618s 4ms/step - loss: 0.0097 - acc: 0.9961 - val_loss: 0.0196 - val_acc: 0.9930

Epoch 00040: val_loss did not improve from 0.01686
Epoch 41/45
139320/139320 [==============================] - 618s 4ms/step - loss: 0.0106 - acc: 0.9959 - val_loss: 0.0211 - val_acc: 0.9938

Epoch 00041: val_loss did not improve from 0.01686
Epoch 42/45
139320/139320 [==============================] - 618s 4ms/step - loss: 0.0117 - acc: 0.9949 - val_loss: 0.0535 - val_acc: 0.9887

Epoch 00042: val_loss did not improve from 0.01686
Epoch 43/45
139320/139320 [====================


Epoch 00030: val_loss did not improve from 0.01635
Epoch 31/45
139320/139320 [==============================] - 619s 4ms/step - loss: 0.0123 - acc: 0.9949 - val_loss: 0.0271 - val_acc: 0.9912

Epoch 00031: val_loss did not improve from 0.01635
Epoch 32/45
139320/139320 [==============================] - 619s 4ms/step - loss: 0.0131 - acc: 0.9952 - val_loss: 0.0175 - val_acc: 0.9942

Epoch 00032: val_loss did not improve from 0.01635
Epoch 33/45
139320/139320 [==============================] - 619s 4ms/step - loss: 0.0110 - acc: 0.9958 - val_loss: 0.0445 - val_acc: 0.9725

Epoch 00033: val_loss did not improve from 0.01635
Epoch 34/45
139320/139320 [==============================] - 619s 4ms/step - loss: 0.0116 - acc: 0.9953 - val_loss: 0.0177 - val_acc: 0.9942

Epoch 00034: val_loss did not improve from 0.01635
Epoch 35/45
139320/139320 [==============================] - 619s 4ms/step - loss: 0.0117 - acc: 0.9956 - val_loss: 0.0253 - val_acc: 0.9912

Epoch 00035: val_loss did not impr

In [11]:

#print(Kmodel.metrics_names)
#print(Kscores)
#valcvscores1=np.array[valcvscores]
print( valcvscores)
print( testcvscores)

[[0.02999345354218263, 0.9887787407769097], [0.07723932808133152, 0.9787333713107639]]
[[0.029805682573560626, 0.9890773071289063], [0.07482893347367645, 0.9793318725585938]]


In [13]:
nowtime = '0321_0205'
from openpyxl import load_workbook
    # convert the history.history dict to a pandas DataFrame:     
    #khist_df = pd.DataFrame(khistor.history) 
    # save to csv: 
if os.path.isfile(rf_path+'/'+nowtime+'_history.xlsx'):
    book = load_workbook(rf_path+'/'+nowtime+'_history.xlsx')     
    writer = pd.ExcelWriter(rf_path+'/'+nowtime+'_history.xlsx', engine='openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    
    khist_df.to_excel(writer,str(k)+'_flod')
    writer.save()
    print('finish')
    

finish


In [12]:
del trainX,trainY, valX, valY
del Kmodel,optim,lr_reducer,callbacks

In [11]:
'''
fp = open(rf_path+'/Result_'+nowtime+'.txt','a')
print("\n",file=fp)
print("acc",file=fp)
print("mean:"+str(np.mean(valcvscores[1,:])*100),file=fp)
print("std:"+str(np.std(valcvscores[1,:])),file=fp)
print("loss",file=fp)
print("mean:"+str(np.mean(valcvscores[0,:])),file=fp)
print("std:"+str(np.std(valcvscores[0,:])),file=fp)

print("\n",file=fp)
print("test",file=fp)
print("mean:"+str(np.mean(testcvscores[1,:])*100),file=fp)
print("std:"+str(np.std(testcvscores[1,:])),file=fp)
print("loss",file=fp)
print("mean:"+str(np.mean(testcvscores[0,:])),file=fp)
print("std:"+str(np.std(testcvscores[0,:])),file=fp)
'''

# convert the history.history dict to a pandas DataFrame:     
cvsval_df = pd.DataFrame(valcvscores,columns=['loss','acc']) 
cvstest_df = pd.DataFrame(testcvscores,columns=['loss','acc']) 
# or save to csv:
writercvscore  = pd.ExcelWriter(rf_path+'/'+nowtime+'_cvscore.xlsx')
cvsval_df.to_excel(writercvscore,'val')
cvstest_df.to_excel(writercvscore,'test')
fp.close()
writer.close()                              
writercvscore.close()
print('Train model finish')

Train model finish


In [ ]:
## auto split ##
epochs = 45 #50
batch_size = 64 #64 #128
optim_type = 'adam'
learning_rate = 0.02 # 0.05 # 0.01 #0.02
sum_time = 0
net = 'unet'
block = '5'

if (net == 'unet')and(block == '5'):
    model = unet_model.ZF_UNET_224(subseq=subseq, filters=32, INPUT_CHANNELS=N_FEATURES, OUTPUT_MASK_CHANNELS=classes)
elif (net == 'unet')and(block == '4'):
    model = unet_model.ZF_UNET_224_4(subseq=subseq, filters=32, INPUT_CHANNELS=N_FEATURES, OUTPUT_MASK_CHANNELS=classes)
elif (net == 'unet')and(block == '3'):
    model = unet_model.ZF_UNET_224_3(subseq=subseq, filters=32, INPUT_CHANNELS=N_FEATURES,
                                     OUTPUT_MASK_CHANNELS=classes)
elif (net == 'unet')and(block == '2'):
    model = unet_model.ZF_UNET_224_2(subseq=subseq, filters=32, INPUT_CHANNELS=N_FEATURES,
                                     OUTPUT_MASK_CHANNELS=classes)
elif (net == 'segnet')and(block == '5'):
    model = segnet.segnet(subseq=subseq, INPUT_CHANNELS=N_FEATURES, filters=64, n_labels=classes, kernel=3,
                          pool_size=(1, 2))
elif net == 'fcn':
    model = unet_model.FCN(inputsize=subseq, deconv_output_size=subseq, INPUT_CHANNELS=N_FEATURES,
                           num_classes=classes)
elif net == 'maskrcnn':
    model = maskrcnn.Mask(subseq=28, INPUT_CHANNELS=N_FEATURES, filters=32, n_labels=classes, kernel=3)

# model = segnet.segnet(subseq=subseq, INPUT_CHANNELS=N_FEATURES,filters=64, n_labels = act_classes, kernel=3, pool_size=(1, 2))
# model = segnet.segnet4(subseq=subseq, INPUT_CHANNELS=N_FEATURES,filters=64, n_labels = act_classes, kernel=3, pool_size=(1, 2))
# model = segnet.segnet3(subseq=subseq, INPUT_CHANNELS=N_FEATURES,filters=64, n_labels = act_classes, kernel=3, pool_size=(1, 2))
# model = segnet.segnet2(subseq=subseq, INPUT_CHANNELS=N_FEATURES,filters=64, n_labels = act_classes, kernel=3, pool_size=(1, 2))

# model = maskrcnn.Mask(subseq=28, INPUT_CHANNELS=N_FEATURES, filters=32, n_labels=act_classes,kernel=3)

# model = unet_model.FCN(inputsize=subseq,deconv_output_size=subseq,INPUT_CHANNELS=N_FEATURES,num_classes=act_classes)
# model = unet_model.ZF_UNET_224_3(subseq=subseq,filters=32, INPUT_CHANNELS=N_FEATURES, OUTPUT_MASK_CHANNELS=act_classes)

In [ ]:
if optim_type == 'SGD':
    optim = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
else:
    optim = Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                            factor=np.sqrt(0.1),
                            cooldown=0,
                            patience=10, min_lr=1e-12)

In [ ]:
# checkpoint
filepath=rf_path+'/'+nowtime+'_'+"weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss',mode='min',verbose=1, save_best_only=True)   # save best model
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]  
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, mode='auto',baseline=None)
callbacks = [lr_reducer, early_stopper, checkpoint]

In [ ]:
print("Model fitting")
history=model.fit(X_train,  Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks)
print("Model fitting is finished")
acc=np.array(history.history['acc'])
loss=np.array(history.history['loss'])
val_acc = np.array(history.history['val_acc'])
val_loss = np.array(history.history['val_loss'])
for i in range(acc.shape[0]):
    logging.info('Epoch: {} loss: {:.4f} accuracy{:.4f} val_loss: {:.4f} val_accuracy{:.4f}%\n'.format(i+1, loss[i], acc[i],val_loss[i],val_acc[i]))

In [ ]:
plt.plot(acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['acc_train', 'acc_val'], loc='upper left')
plt.savefig(rf_path+'/'+nowtime+'_'+str(subseq)+'_'+dbName+'_U Net_model_acc_'+ str(batch_size)+'.png')
plt.show()

plt.figure(30)
plt.plot(loss)
plt.plot(val_loss)
plt.title('U net Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['loss_train','loss_val'], loc='upper left')
plt.savefig(rf_path+'/'+nowtime+'_'+str(subseq)+'_'+dbName+'_U Net_model_loss_'+ str(batch_size)+'.png')
plt.show()

''''''

In [ ]:
start = time.clock()
testloss, testaccuracy = model.evaluate(X_test, Y_test, batch_size=batch_size)
end = time.clock()
sum_time += (end - start)
print(str(end - start))
print("test lost:" + str(testloss) + "   test accuracy:" + str(testaccuracy),file=fp)
logging.info('mean_time={}'.format(str(end - start)))


y_test_resh = Y_test.reshape(Y_test.shape[0], Y_test.shape[2], -1)
y_test_resh_argmax = np.argmax(y_test_resh, axis=2)
labels_test_unary = y_test_resh_argmax.reshape(y_test_resh_argmax.size)
file_labels_test_unary = rf_path+'/'+nowtime +'_labels_gd_'+dbName+'_'+str(subseq)+'_'+ net+ str(block)+'.npy'
np.save(file_labels_test_unary,labels_test_unary)

y_pred_raw = model.predict(X_test, batch_size=batch_size)
y_pred_resh = y_pred_raw.reshape(y_pred_raw.shape[0], y_pred_raw.shape[2], -1)
y_pred_resh_argmax = np.argmax(y_pred_resh, axis=2)
y_pred = y_pred_resh_argmax.reshape(y_pred_resh_argmax.size)
y_pred_prob = y_pred_resh.reshape(y_pred_resh_argmax.size,y_pred_resh.shape[2])
print("---------------------------",file=fp)
print(y_pred_prob.shape,file=fp)
file_y_pred =rf_path+'/'+nowtime+ '_y_pred_'+dbName+'_'+str(subseq)+'_'+net+'_'+block+'_'+optim_type+'.npy'
np.save(file_y_pred,y_pred)
file_y_pred_prob = rf_path+'/'+nowtime+'_y_pred_prob_'+dbName+'_'+str(subseq)+'_'+net+'_'+block+'_'+optim_type+'.npy'
np.save(file_y_pred_prob,y_pred_prob)

In [ ]:
print("Mean absolute error (MAE):      %f" % sklearn.metrics.mean_absolute_error(labels_test_unary,y_pred),file=fp)
print("Mean squared error (MSE):       %f" % sklearn.metrics.mean_squared_error(labels_test_unary,y_pred),file=fp)
print("Root mean squared error (RMSE): %f" % math.sqrt(sklearn.metrics.mean_squared_error(labels_test_unary,y_pred)),file=fp)
print("R square (R^2):                 %f" % sklearn.metrics.r2_score(labels_test_unary,y_pred),file=fp)

In [ ]:
print(nowtime,file=fp)
print('Test before saves:', model.predict(X_test[0:2]),file=fp)
file_model = rf_path+'/Model_'+nowtime + '_'+ str(subseq) +'_' + str(batch_size) + '_' + optim_type + '.h5py'
model.save(file_model)
file_weight = rf_path+'/Weight_'+nowtime + '_'+ str(subseq) +'_' + str(batch_size) + '_' + optim_type + '.h5py'
model.save_weights(file_weight)
fp.close()
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = rf_path+'/'+nowtime+'_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)